In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers
from tqdm import trange

## 1. Read + prepare data
Data is expected to be transformed into a single-step version using the `transform_data_to_single_step_workflow.py` script.

In [2]:
df = pd.read_csv('data/single_step_df_ints_2022-10-11_encoded.csv', header=0)

Float columns: Fill NAs with -99.

In [3]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

Object columns: Fill NAs with empty string.

In [4]:
cols = df.select_dtypes(include=['object'])
for col in cols.columns.values:
    df[col] = df[col].fillna('')

Show value counts for target to check if all classes have more than one instance (required for stratisfied shuffle split).

In [5]:
df['target'].value_counts()
df[:1437]['target'].value_counts()

p    188
k    162
g    160
f    157
c    148
s    134
b     78
w     74
o     73
a     72
r     47
d     27
h     27
t     22
e     22
z     21
m     13
n      9
x      2
i      1
Name: target, dtype: int64

Remove row with class that only occurs once.

In [6]:
index_to_drop = df['target'].loc[df['target'] == 'i'].index[0]
index_to_drop

727

In [7]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [8]:
df_new = df.drop(index=[index_to_drop], axis=0)
df_new.reset_index(inplace=True, drop=True)

Use stratisfied shuffle split to generate training and test set. The dataframe is cut at index 1436 here to only use the table setting data for training the model.

In [9]:
split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_new[:1436], df_new[:1436]['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

Split test data into test and validation set.

In [10]:
split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

Generate list of labels to pass to MultiLabelBinarizer so there's the same number of classes for all datasets.

In [11]:
labels = df_new['target'].unique()

In [12]:
len(labels)

35

## 2. Create train, test, validation data sets

In [13]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [14]:
batch_size = 128

In [15]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size);

/tmp/ipykernel_86683/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}
2023-08-08 12:07:29.428005: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-08 12:07:29.460784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-08 12:07:29.460923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-08 12:07:29.461463: I tensorflow/core/platform/c

## 3. Define functions to create layers and input data

In [16]:
def get_normalization_layer(name, dataset):
  # create a normalization layer for the feature
    normalizer = layers.Normalization(axis=None)

  # prepare a dataset that only yields the feature
    feature_ds = dataset.map(lambda x, y: x[name])

  # learn the statistics of the data
    normalizer.adapt(feature_ds)
    
    return normalizer

In [17]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [18]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

## 4. Create and train model

In [19]:
all_inputs, encoded_features = create_input_data(df)

In [20]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [21]:
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(len(labels))(x)

model = tf.keras.Model(all_inputs, output)

In [22]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [23]:
model.fit(train_ds, epochs=300, validation_data=val_ds, callbacks=[callback])

Epoch 1/300
8/8 [==============================] - 12s 633ms/step - loss: 3.2333 - accuracy: 0.1144 - val_loss: 15577.4580 - val_accuracy: 0.2083
Epoch 2/300
8/8 [==============================] - 1s 69ms/step - loss: 2.5568 - accuracy: 0.1751 - val_loss: 15611.1729 - val_accuracy: 0.2130
Epoch 3/300
8/8 [==============================] - 1s 68ms/step - loss: 2.3916 - accuracy: 0.2000 - val_loss: 15490.7266 - val_accuracy: 0.1991
Epoch 4/300
8/8 [==============================] - 1s 69ms/step - loss: 2.2532 - accuracy: 0.2299 - val_loss: 15001.9697 - val_accuracy: 0.3426
Epoch 5/300
8/8 [==============================] - 1s 68ms/step - loss: 2.1739 - accuracy: 0.2169 - val_loss: 16203.2803 - val_accuracy: 0.3565
Epoch 6/300
8/8 [==============================] - 1s 69ms/step - loss: 2.0505 - accuracy: 0.2667 - val_loss: 16357.5703 - val_accuracy: 0.3519
Epoch 7/300
8/8 [==============================] - 1s 69ms/step - loss: 1.9700 - accuracy: 0.3025 - val_loss: 17402.9062 - val_accurac

8/8 [==============================] - 1s 69ms/step - loss: 1.1027 - accuracy: 0.6000 - val_loss: 26222.0176 - val_accuracy: 0.5694
Epoch 58/300
8/8 [==============================] - 1s 70ms/step - loss: 1.1011 - accuracy: 0.6060 - val_loss: 24907.6855 - val_accuracy: 0.5787
Epoch 59/300
8/8 [==============================] - 1s 71ms/step - loss: 1.0568 - accuracy: 0.6398 - val_loss: 25697.3145 - val_accuracy: 0.5787
Epoch 60/300
8/8 [==============================] - 1s 70ms/step - loss: 1.0885 - accuracy: 0.5801 - val_loss: 25889.1250 - val_accuracy: 0.5278
Epoch 61/300
8/8 [==============================] - 1s 70ms/step - loss: 1.0672 - accuracy: 0.6080 - val_loss: 25287.9492 - val_accuracy: 0.5880
Epoch 62/300
8/8 [==============================] - 1s 69ms/step - loss: 1.0437 - accuracy: 0.6209 - val_loss: 25722.9453 - val_accuracy: 0.5648
Epoch 63/300
8/8 [==============================] - 1s 72ms/step - loss: 1.0686 - accuracy: 0.6179 - val_loss: 25904.4824 - val_accuracy: 0.574

In [24]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 44ms/step - loss: 21389.1230 - accuracy: 0.5163
Accuracy 0.5162790417671204


Optional: Save model for later reuse.

In [25]:
# model.save('models/next_obj_classifier_tablesetting')

In [26]:
# reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_tablesetting')

### Test model prediction for one sample

In [27]:
sample = df.loc[0].drop('target').to_dict()

In [28]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [29]:
prediction = model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

Get label for prediction.

In [30]:
pred_label = labels[np.argmax(prediction)]
pred_label

'p'

## 5. Run prediction for each sequence using prequential approach

In [31]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    predictions = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in trange(dataframe.index[0], dataframe.index[-1]): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        predictions[sequence_nr].append(pred_label)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors, predictions

Define number of sequences (- 1) for which prediction is run. The test run is done on the cleaning up data set, which starts at index 2075.

In [32]:
nr_of_sequences_preds = len(df[2075:][df[2075:]['input'] == '<start>'])
nr_of_sequences_preds

16

Run prediction, sum up errors and get median.

In [33]:
errors, predictions = get_prequential_error(df[2075:], model, labels, nr_of_sequences_preds)

100%|██████████████████████████████████████████████████████████████| 127/127 [01:24<00:00,  1.50it/s]


In [34]:
summed_error = [sum(error) for error in errors]

In [35]:
np.median(summed_error)

8.0